In [50]:
import random
import networkx as nx
import pandas as pd
f = pd.read_csv("higgs-activity_time.txt.gz",sep=" ",names=['source','target','timestamp','type'])
d = pd.read_csv("higgs-social_network.edgelist.gz",sep=" ",names=['source','target'])

g = nx.from_pandas_edgelist(f[:10000],source="source",target="target",edge_attr = ["timestamp","type"])

In [51]:
g_l = list(g.edges(data=True))
g_l = sorted(g_l,key = lambda x: x[2]['timestamp'])

In [52]:
#g_neigh = pd.DataFrame(d.groupby('source')['target'].apply(list))

In [53]:
g_n = d.groupby('source').count().to_dict(orient='dict')['target']
for i in g_n.keys():
    g_n[i] = [g_n[i],"inactive",0,0,False,0,set(),-1]

In [54]:
g_n[1]

[22, 'inactive', 0, 0, False, 0, set(), -1]

In [55]:
g_l[1]

(26375, 168366, {'timestamp': 1341101183, 'type': 'MT'})

In [56]:
graph = {'links' : g_l, 'nodes' : g_n}

In [57]:
def LT(graph):
    active = []
    blocked = []
    count = 0
    for i in graph['links']:
        if graph['nodes'][i[0]][4] != True:
            graph['nodes'][i[0]][1] = "active"
            graph['nodes'][i[0]][2] = 15
            graph['nodes'][i[0]][3] = i[2]['timestamp']
            
        for j in blocked:
            if graph['nodes'][j][5] - 1 == 0:
                graph['nodes'][j][4] = False
                graph['nodes'][j][5] = 0
                blocked.remove(j)
            else:
                graph['nodes'][j][5] -= 1
        
        for j in active:
            if graph['nodes'][j][2] - 1 == 0:
                graph['nodes'][j][2] = 0
                graph['nodes'][j][1] = "inactive"
                active.remove(j)
            else:
                graph['nodes'][j][2] -= 1
                if i[2]['timestamp'] - graph['nodes'][j][3] > 10000:
                    #print(f"Rumor propogation minimized at {i[2]['timestamp']} by node {j}")
                    graph['nodes'][j][1] = "inactive"
                    active.remove(j)
                    r_node = graph['nodes'][j][7]
                    if graph['nodes'][r_node][4] == False:
                        temp_count = 0
                        for temp in graph['nodes'][r_node][6]:
                            if (graph['nodes'][temp][1] == "inactive" or (i[2]['timestamp'] - graph['nodes'][temp][3] > 7000)) and graph['nodes'][temp][2] > 0:
                                temp_count += 1
                        if temp_count > 0.9*len(graph['nodes'][r_node][6]):
                            graph['nodes'][r_node][4] = True
                            graph['nodes'][r_node][5] = 200
                            blocked.append(r_node)
                            print(f"Rumor spreading node {r_node} blocked at {i[2]['timestamp']}")
                            count += 1
                    else:
                        pass
                else:
                    pass
            
        graph['nodes'][i[0]][6].add(i[1])
        
        if graph['nodes'][i[0]][4] == True:
            pass
        else:   
            graph['nodes'][i[0]][1] = "active"
            if graph['nodes'][i[1]][1] == "inactive":
                if 1/graph['nodes'][i[0]][0] < 1/graph['nodes'][i[1]][0]:
                    graph['nodes'][i[1]][1] = "active"
                    graph['nodes'][i[1]][2] = 200
                    graph['nodes'][i[1]][3] = i[2]['timestamp']
                    graph['nodes'][i[1]][7] = i[0]
                    active.append(i[1])
            else:
                pass
            
    print(count)

In [58]:
LT(graph)

Rumor spreading node 75083 blocked at 1341111775
Rumor spreading node 104321 blocked at 1341112823
Rumor spreading node 247125 blocked at 1341113457
Rumor spreading node 205864 blocked at 1341113457
Rumor spreading node 64911 blocked at 1341114502
Rumor spreading node 2733 blocked at 1341114733
Rumor spreading node 149403 blocked at 1341115105
Rumor spreading node 64375 blocked at 1341115621
Rumor spreading node 197061 blocked at 1341116534
Rumor spreading node 4665 blocked at 1341116605
Rumor spreading node 33833 blocked at 1341117253
Rumor spreading node 162386 blocked at 1341118325
Rumor spreading node 82732 blocked at 1341119256
Rumor spreading node 8 blocked at 1341120214
Rumor spreading node 321340 blocked at 1341120214
Rumor spreading node 107640 blocked at 1341121164
Rumor spreading node 3024 blocked at 1341121239
Rumor spreading node 184615 blocked at 1341121409
Rumor spreading node 10367 blocked at 1341121546
Rumor spreading node 133104 blocked at 1341122771
Rumor spreading n